In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from keras.regularizers import l1_l2
from sklearn.model_selection import cross_val_score, KFold
from keras.callbacks import EarlyStopping
from tqdm import tqdm
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.utils import class_weight
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score


# Load the data
data_path = './datasets/group47/dataset/intermediate_datafiles/chapter5_group47_result.csv'
data = pd.read_csv(data_path, index_col=0)

# Define the target columns
target_columns = ['labelCycling', 'labelStairs', 'labelWalking', 'labelSitting', 'labelOther']

# Separate the features and target variables
X = data.drop(target_columns, axis=1).fillna(0).replace([np.inf], np.finfo('float64').max).replace([-np.inf], np.finfo('float64').min)
y = data[target_columns]

selected_features = set()
for column in tqdm(y.columns, desc="Selecting features"):
    y_single_label = LabelEncoder().fit_transform(y[column])  # Convert to 1D array
    selector = SelectKBest(mutual_info_classif, k=50)
    X_new = selector.fit_transform(X, y_single_label)
    selected_features.update(X.columns[selector.get_support()])

# Filter the features based on the selected features
X = StandardScaler().fit_transform(X[list(selected_features)])

# Use simple random sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Define a function to create and compile a new model
def create_model(learning_rate=0.001, dropout_rate=0.2, neurons=100):
    model = Sequential()
    model.add(LSTM(neurons, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate, kernel_regularizer=l1_l2(l1=0.01, l2=0.01), input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(neurons, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate, kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
    model.add(LSTM(neurons, return_sequences=False, dropout=dropout_rate, recurrent_dropout=dropout_rate, kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
    model.add(Dense(neurons, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
    model.add(Dense(len(target_columns), activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])
    return model

# Create a new model
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
learning_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
neurons = [50, 100, 150, 200]
param_grid = dict(learning_rate=learning_rate, dropout_rate=dropout_rate, neurons=neurons)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, verbose=1)
grid_result = grid.fit(X_train, y_train)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Define the cross validation iterator
kfold = KFold(n_splits=10, shuffle=True)

# Perform cross validation
results = cross_val_score(model, X_train, y_train, cv=kfold)


# Print the cross validation score
print(f"Cross Validation Accuracy: {results.mean()} (+/- {results.std() * 2})")




Selecting features: 100%|██████████| 5/5 [00:25<00:00,  5.17s/it]
C:\Users\patri\AppData\Local\Temp\ipykernel_37160\3267065050.py:62: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed: 35.1min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 85.4min finished


Best: 0.526977 using {'dropout_rate': 0.0, 'learning_rate': 0.001, 'neurons': 50}
Cross Validation Accuracy: 0.52696493268013 (+/- 0.09956798486805307)


In [2]:
# Get the best parameters for LSTM model
best_params_lstm = grid_result.best_params_

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=50)

# Create and compile the LSTM model with the best parameters
model = create_model(learning_rate=best_params_lstm['learning_rate'], 
                     dropout_rate=best_params_lstm['dropout_rate'], 
                     neurons=best_params_lstm['neurons'])

In [8]:
# Fit the model with the training data
model.fit(X_train, y_train, validation_split=0.2, epochs=100, callbacks=[early_stopping], verbose=1)

Epoch 1/100
61/61 [==============================] - 5s 60ms/step - loss: 0.4535 - accuracy: 0.5265 - val_loss: 0.4433 - val_accuracy: 0.5288
Epoch 2/100
61/61 [==============================] - 3s 48ms/step - loss: 0.4428 - accuracy: 0.5265 - val_loss: 0.4395 - val_accuracy: 0.5288
Epoch 3/100
61/61 [==============================] - 3s 47ms/step - loss: 0.4404 - accuracy: 0.5265 - val_loss: 0.4376 - val_accuracy: 0.5288
Epoch 4/100
61/61 [==============================] - 3s 48ms/step - loss: 0.4406 - accuracy: 0.5265 - val_loss: 0.4387 - val_accuracy: 0.5288
Epoch 5/100
61/61 [==============================] - 3s 47ms/step - loss: 0.4403 - accuracy: 0.5265 - val_loss: 0.4382 - val_accuracy: 0.5288
Epoch 6/100
61/61 [==============================] - 3s 49ms/step - loss: 0.4409 - accuracy: 0.5265 - val_loss: 0.4390 - val_accuracy: 0.5288
Epoch 7/100
61/61 [==============================] - 3s 47ms/step - loss: 0.4401 - accuracy: 0.5265 - val_loss: 0.4385 - val_accuracy: 0.5288
Epoch 

In [9]:
# Make predictions on the testing data
y_pred_bin = (model.predict(X_test) > 0.5).astype(int)

# Print the classification report
print(classification_report(y_test, y_pred_bin, target_names=target_columns))

# Print precision, recall,and F1 score
print(f"Precision: {precision_score(y_test, y_pred_bin, average='weighted', zero_division=1)}")
print(f"Recall: {recall_score(y_test, y_pred_bin, average='weighted', zero_division=1)}")
print(f"F1 Score: {f1_score(y_test, y_pred_bin, average='weighted', zero_division=1)}")

38/38 [==============================] - 1s 15ms/step
              precision    recall  f1-score   support

labelCycling       0.00      0.00      0.00       247
 labelStairs       0.00      0.00      0.00        64
labelWalking       0.00      0.00      0.00       192
labelSitting       0.54      1.00      0.70       649
  labelOther       0.00      0.00      0.00        50

   micro avg       0.54      0.54      0.54      1202
   macro avg       0.11      0.20      0.14      1202
weighted avg       0.29      0.54      0.38      1202
 samples avg       0.54      0.54      0.54      1202

Precision: 0.7530571956438267
Recall: 0.5399334442595674
F1 Score: 0.3798556155278691


c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
# Compute class weights
class_weights_list = []
for column in y_train.columns:
    class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train[column]), y_train[column])
    class_weights_dict = dict(enumerate(class_weights))
    class_weights_list.append(class_weights_dict)

In [10]:
def create_model():
    model = Sequential()
    model.add(LSTM(200, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(100, return_sequences=False, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(len(target_columns), activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'], weighted_metrics=['accuracy'])
    return model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'], weighted_metrics=[])


In [11]:
def calculate_sample_weights(y_train, class_weights_list):
    sample_weights = []
    for i, row in y_train.iterrows():
        weights = [class_weights_list[j][row[j]] for j in range(len(row))]
        sample_weights.append(np.mean(weights))
    return np.array(sample_weights)

sample_weights = calculate_sample_weights(y_train, class_weights_list)

model.fit(X_train, y_train, validation_split=0.2, epochs=100, callbacks=[early_stopping], sample_weight=sample_weights, verbose=1)


Epoch 1/100
61/61 [==============================] - 7s 59ms/step - loss: 0.5394 - accuracy: 0.5265 - val_loss: 0.5104 - val_accuracy: 0.5288
Epoch 2/100
61/61 [==============================] - 3s 44ms/step - loss: 0.5110 - accuracy: 0.5265 - val_loss: 0.5055 - val_accuracy: 0.5288
Epoch 3/100
61/61 [==============================] - 3s 45ms/step - loss: 0.5089 - accuracy: 0.5265 - val_loss: 0.5049 - val_accuracy: 0.5288
Epoch 4/100
61/61 [==============================] - 3s 44ms/step - loss: 0.5086 - accuracy: 0.5265 - val_loss: 0.5059 - val_accuracy: 0.5288
Epoch 5/100
61/61 [==============================] - 3s 43ms/step - loss: 0.5088 - accuracy: 0.5265 - val_loss: 0.5117 - val_accuracy: 0.5288
Epoch 6/100
61/61 [==============================] - 3s 43ms/step - loss: 0.5096 - accuracy: 0.5265 - val_loss: 0.5100 - val_accuracy: 0.5288
Epoch 7/100
61/61 [==============================] - 3s 43ms/step - loss: 0.5096 - accuracy: 0.5265 - val_loss: 0.5014 - val_accuracy: 0.5288
Epoch 

In [12]:
# Make predictions on the testing data
y_pred_bin = (model.predict(X_test) > 0.5).astype(int)

# Print the classification report
print(classification_report(y_test, y_pred_bin, target_names=target_columns))

# Print precision, recall,and F1 score
print(f"Precision: {precision_score(y_test, y_pred_bin, average='weighted', zero_division=1)}")
print(f"Recall: {recall_score(y_test, y_pred_bin, average='weighted', zero_division=1)}")
print(f"F1 Score: {f1_score(y_test, y_pred_bin, average='weighted', zero_division=1)}")

38/38 [==============================] - 2s 15ms/step
              precision    recall  f1-score   support

labelCycling       0.00      0.00      0.00       247
 labelStairs       0.00      0.00      0.00        64
labelWalking       0.00      0.00      0.00       192
labelSitting       0.00      0.00      0.00       649
  labelOther       0.00      0.00      0.00        50

   micro avg       0.00      0.00      0.00      1202
   macro avg       0.00      0.00      0.00      1202
weighted avg       0.00      0.00      0.00      1202
 samples avg       0.00      0.00      0.00      1202

Precision: 1.0
Recall: 0.0
F1 Score: 0.0


c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patri\.conda\en

In [13]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)
roc_auc = roc_auc_score(y_test, y_pred, average='weighted')


print('Precision: ', precision)
print('Recall: ', recall)
print('F1 score: ', f1)
print('ROC AUC: ', roc_auc)

38/38 [==============================] - 1s 14ms/step
Precision:  1.0
Recall:  0.0
F1 score:  0.0
ROC AUC:  0.5
